In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from matplotlib import image
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ModelCheckpoint
 
plt.rcParams['figure.figsize'] = (5,5)
plt.rcParams['figure.dpi'] = 150
 
tf.random.set_seed(42)
np.random.RandomState(42)
 
    
%load_ext autoreload
%autoreload 2

In [2]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('train/ER') if isfile(join('train/ER', f))]

names_ER = list(map(lambda x: x.strip('.png'), onlyfiles))

In [3]:
ER = []
for k in tqdm(onlyfiles):
    img = tf.keras.preprocessing.image.load_img(
        'train/ER/'+k, color_mode='grayscale', target_size=(576,576))
    input_arr = tf.keras.preprocessing.image.img_to_array(img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    ER.append(input_arr)
ER = np.asarray(ER)

100%|██████████| 6758/6758 [00:29<00:00, 226.61it/s]


In [4]:
onlyfiles_NR = [f for f in listdir('train/NR') if isfile(join('train/NR', f))]

names_NR = list(map(lambda x: x.strip('.png'), onlyfiles))

In [5]:
NR = []
for k in tqdm(onlyfiles_NR):
    img = tf.keras.preprocessing.image.load_img(
        'train/NR/'+k, color_mode='grayscale', target_size=(576,576))
    input_arr = tf.keras.preprocessing.image.img_to_array(img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    NR.append(input_arr)
NR = np.asarray(NR)

100%|██████████| 6646/6646 [00:29<00:00, 227.88it/s]


In [6]:
ER = ER.reshape(-1,576,576)
NR = NR.reshape(-1,576,576)

In [7]:
ER = ER[:, 225:-225, 225:-225]
NR = NR[:, 225:-225, 225:-225]

In [8]:
y_ER = np.zeros(ER.shape[0])
y_NR = np.ones(NR.shape[0])

In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(np.concatenate([ER, NR], axis=0), 
                                                np.concatenate([y_ER, y_NR], axis=0), shuffle=True, 
                                               random_state=42)

In [10]:
Xtrain.shape

(10053, 126, 126)

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()

In [13]:
Xtrain.reshape(-1).shape

(159601428,)

In [14]:
Xtrain = scaler.fit_transform(Xtrain.reshape(-1, 1)).reshape(-1,126,126,1)
Xtest = scaler.transform(Xtest.reshape(-1, 1)).reshape(-1,126,126,1)

In [21]:
# model using functional /keras API instead of sequential API
inputs = keras.Input(shape=(126,126,1))

In [22]:
x = layers.Conv2D(30, 3, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(100, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

In [23]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [24]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 126, 126, 1)]     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 30)      300       
_________________________________________________________________
batch_normalization_1 (Batch (None, 124, 124, 30)      120       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 30)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 115320)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 115320)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               1153

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.losses.BinaryCrossentropy(from_logits=False),
  metrics=['accuracy', tf.keras.metrics.AUC()])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
checkpoint = ModelCheckpoint("MLHEP_v8_weights/weights.{epoch:02d}-{val_loss:.4f}.hdf5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq='epoch')

In [26]:
history = model.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), epochs=250, shuffle=True, 
                    callbacks=[checkpoint])

Epoch 1/250
315/315 [==============================] - 4s 10ms/step - loss: 1.0471 - accuracy: 0.6598 - auc_1: 0.7071 - val_loss: 4.1700 - val_accuracy: 0.5112 - val_auc_1: 0.5000

Epoch 00001: accuracy improved from -inf to 0.77639, saving model to MLHEP_v8_weights/weights.01-4.1700.hdf5
Epoch 2/250
315/315 [==============================] - 3s 9ms/step - loss: 0.1310 - accuracy: 0.9580 - auc_1: 0.9897 - val_loss: 5.4129 - val_accuracy: 0.5112 - val_auc_1: 0.4997

Epoch 00002: accuracy improved from 0.77639 to 0.96429, saving model to MLHEP_v8_weights/weights.02-5.4129.hdf5
Epoch 3/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0503 - accuracy: 0.9870 - auc_1: 0.9980 - val_loss: 6.4066 - val_accuracy: 0.5112 - val_auc_1: 0.5000

Epoch 00003: accuracy improved from 0.96429 to 0.98558, saving model to MLHEP_v8_weights/weights.03-6.4066.hdf5
Epoch 4/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0420 - accuracy: 0.9886 - auc_1: 0.9982 - val_los

315/315 [==============================] - 3s 9ms/step - loss: 0.0075 - accuracy: 0.9992 - auc_1: 0.9996 - val_loss: 0.0681 - val_accuracy: 0.9860 - val_auc_1: 0.9953

Epoch 00033: accuracy did not improve from 0.99851
Epoch 34/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0028 - accuracy: 0.9993 - auc_1: 1.0000 - val_loss: 0.0044 - val_accuracy: 0.9982 - val_auc_1: 1.0000

Epoch 00034: accuracy improved from 0.99851 to 0.99910, saving model to MLHEP_v8_weights/weights.34-0.0044.hdf5
Epoch 35/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0076 - accuracy: 0.9980 - auc_1: 0.9996 - val_loss: 1.2541 - val_accuracy: 0.8535 - val_auc_1: 0.9003

Epoch 00035: accuracy did not improve from 0.99910
Epoch 36/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0067 - accuracy: 0.9984 - auc_1: 0.9998 - val_loss: 0.1276 - val_accuracy: 0.9818 - val_auc_1: 0.9928

Epoch 00036: accuracy did not improve from 0.99910
Epoch 37/250
315/315 [==

315/315 [==============================] - 3s 9ms/step - loss: 0.0047 - accuracy: 0.9989 - auc_1: 0.9995 - val_loss: 1.3400 - val_accuracy: 0.9406 - val_auc_1: 0.9530

Epoch 00102: accuracy did not improve from 0.99980
Epoch 103/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0017 - accuracy: 0.9997 - auc_1: 0.9998 - val_loss: 0.0051 - val_accuracy: 0.9994 - val_auc_1: 0.9997

Epoch 00103: accuracy did not improve from 0.99980
Epoch 104/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0084 - accuracy: 0.9981 - auc_1: 0.9997 - val_loss: 18.5786 - val_accuracy: 0.7067 - val_auc_1: 0.7280

Epoch 00104: accuracy did not improve from 0.99980
Epoch 105/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0139 - accuracy: 0.9980 - auc_1: 0.9990 - val_loss: 0.0716 - val_accuracy: 0.9964 - val_auc_1: 0.9974

Epoch 00105: accuracy did not improve from 0.99980
Epoch 106/250
315/315 [==============================] - 3s 9ms/step - loss: 0.01

315/315 [==============================] - 3s 9ms/step - loss: 0.0020 - accuracy: 0.9994 - auc_1: 0.9999 - val_loss: 0.0090 - val_accuracy: 0.9985 - val_auc_1: 0.9994

Epoch 00172: accuracy did not improve from 0.99990
Epoch 173/250
315/315 [==============================] - 3s 9ms/step - loss: 4.4506e-04 - accuracy: 0.9999 - auc_1: 1.0000 - val_loss: 0.0477 - val_accuracy: 0.9976 - val_auc_1: 0.9982

Epoch 00173: accuracy did not improve from 0.99990
Epoch 174/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0039 - accuracy: 0.9997 - auc_1: 0.9998 - val_loss: 0.0141 - val_accuracy: 0.9991 - val_auc_1: 0.9997

Epoch 00174: accuracy did not improve from 0.99990
Epoch 175/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0049 - accuracy: 0.9994 - auc_1: 0.9998 - val_loss: 0.0154 - val_accuracy: 0.9982 - val_auc_1: 0.9994

Epoch 00175: accuracy did not improve from 0.99990
Epoch 176/250
315/315 [==============================] - 3s 9ms/step - loss: 0

315/315 [==============================] - 3s 9ms/step - loss: 0.0058 - accuracy: 0.9988 - auc_1: 0.9996 - val_loss: 2.1576 - val_accuracy: 0.9418 - val_auc_1: 0.9533

Epoch 00242: accuracy did not improve from 0.99990
Epoch 243/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0013 - accuracy: 0.9998 - auc_1: 1.0000 - val_loss: 0.0632 - val_accuracy: 0.9869 - val_auc_1: 0.9972

Epoch 00243: accuracy did not improve from 0.99990
Epoch 244/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0014 - accuracy: 0.9998 - auc_1: 0.9999 - val_loss: 22.2128 - val_accuracy: 0.5121 - val_auc_1: 0.5037

Epoch 00244: accuracy did not improve from 0.99990
Epoch 245/250
315/315 [==============================] - 3s 9ms/step - loss: 0.0022 - accuracy: 0.9998 - auc_1: 1.0000 - val_loss: 0.0130 - val_accuracy: 0.9991 - val_auc_1: 0.9997

Epoch 00245: accuracy did not improve from 0.99990
Epoch 246/250
315/315 [==============================] - 3s 9ms/step - loss: 0.00

In [27]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('test/pattern') if isfile(join('test/pattern', f))]

In [28]:
names = list(map(lambda x: x.strip('.png'), onlyfiles))

In [36]:
model.load_weights('MLHEP_v8_weights/weights.97-0.0048.hdf5')

In [30]:
predictions_arr =[]
n_id = []
for k in tqdm(onlyfiles):
    img = tf.keras.preprocessing.image.load_img(
        'test/pattern/'+k, color_mode='grayscale', target_size=(576,576))
    input_arr = tf.keras.preprocessing.image.img_to_array(img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    predictions_arr.append(input_arr)
    n_id.append(k.strip('.png'))
predictions_arr = np.asarray(predictions_arr)

100%|██████████| 16560/16560 [01:10<00:00, 236.32it/s]


In [31]:
predictions_arr = predictions_arr.reshape(-1,576,576,1)

predictions_arr = predictions_arr[:, 225:-225, 225:-225]
predictions_arr = scaler.transform(predictions_arr.reshape(-1, 1)).reshape(-1,126,126,1)

In [37]:
predictions = model.predict(predictions_arr)

In [38]:
import pandas as pd

In [39]:
d = {'id': n_id, 'particle': predictions.reshape(-1)}
dataframe = pd.DataFrame(d)
dataframe['particle'] = 1-dataframe['particle']

In [40]:
dataframe.to_csv('submission_v12', index=False)